# Understanding the Transform Function in Pandas
http://pbpython.com/pandas_transform.html

transform is an operation used in conjunction with groupby

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean.

In [1]:
import pandas as pd

In [2]:
# df = pd.read_excel("sales_transactions1.xlsx")
df = pd.read_csv("sales_transactions.csv")

In [3]:
df

,account,name,order,sku,quantity,unit price,ext price
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65


For example, if we look at order 10001 with a total of $576.12, the break down would be:

* B1-20000 = $235.83 or 40.9%

* S1-27722 = $232.32 or 40.3%

* B1-86481 = $107.97 or 18.7%

The tricky part in this calculation is that we need to get a total for each order and combine it back with the transaction level detail in order to get the percentages. In Excel, you could try to use some version of a subtotal to try to calculate the values.

## First Approach - Merging

In [4]:
df.groupby('order')["ext price"].sum()

order
10001     576.12
10005    8185.49
10006    3724.49
Name: ext price, dtype: float64

In [5]:
order_total = df.groupby('order')["ext price"].sum().rename("Order_Total").reset_index()

In [6]:
order_total

,order,Order_Total
0,10001,576.12
1,10005,8185.49
2,10006,3724.49


In [7]:
df_1 = df.merge(order_total)

In [8]:
df_1["Percent_of_Order"] = df_1["ext price"] / df_1["Order_Total"]

In [9]:
df_1

,account,name,order,sku,quantity,unit price,ext price,Order_Total,Percent_of_Order
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83,576.12,0.409342
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32,576.12,0.403249
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97,576.12,0.187409
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36,8185.49,0.327330
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02,8185.49,0.034942
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95,8185.49,0.101759
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04,8185.49,0.424170
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12,8185.49,0.111798
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12,3724.49,0.821890
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65,3724.49,0.139254


## Second Approach - Using Transform

You will notice how this returns a different size data set from our normal groupby functions. Instead of only showing the totals for 3 orders, we retain the same number of items as the original data set. That is the unique feature of using transform .

In [10]:
df.groupby('order')["ext price"].transform('sum')

0      576.12
1      576.12
2      576.12
3     8185.49
4     8185.49
5     8185.49
6     8185.49
7     8185.49
8     3724.49
9     3724.49
10    3724.49
11    3724.49
Name: ext price, dtype: float64

In [11]:
df["Order_Total"] = df.groupby('order')["ext price"].transform('sum')

In [12]:
df["Percent_of_Order"] = df["ext price"] / df.groupby('order')["ext price"].transform('sum')

In [13]:
df

,account,name,order,sku,quantity,unit price,ext price,Order_Total,Percent_of_Order
0,383080,Will LLC,10001,B1-20000,7,33.69,235.83,576.12,0.409342
1,383080,Will LLC,10001,S1-27722,11,21.12,232.32,576.12,0.403249
2,383080,Will LLC,10001,B1-86481,3,35.99,107.97,576.12,0.187409
3,412290,Jerde-Hilpert,10005,S1-06532,48,55.82,2679.36,8185.49,0.327330
4,412290,Jerde-Hilpert,10005,S1-82801,21,13.62,286.02,8185.49,0.034942
5,412290,Jerde-Hilpert,10005,S1-06532,9,92.55,832.95,8185.49,0.101759
6,412290,Jerde-Hilpert,10005,S1-47412,44,78.91,3472.04,8185.49,0.424170
7,412290,Jerde-Hilpert,10005,S1-27722,36,25.42,915.12,8185.49,0.111798
8,218895,Kulas Inc,10006,S1-27722,32,95.66,3061.12,3724.49,0.821890
9,218895,Kulas Inc,10006,B1-33087,23,22.55,518.65,3724.49,0.139254
